In [ ]:
import cv2
import os
import pickle
import numpy as np
from pycocotools.coco import COCO 
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
skeleton = pickle.load(open('coco_skeleton.pkl', 'rb'))

In [ ]:
for coco_split, file_split in [('train', 'train'), ('val', 'val')]:
    coco = COCO('data/mals/annotations/person_keypoints_{}2014.json'.format(coco_split))
    filepath = 'data/skeleton_black/{}/{}'
    files = pickle.load(open('data/mals/{}.pkl'.format(file_split), 'rb'))
    for i in files:
        image = cv2.imread(i)
        image = np.zeros(image.shape, np.uint8)
        imgId = int(i.split('_')[-1].split('.')[0])
        annIds = coco.getAnnIds(imgId, catIds=[1], iscrowd=False)
        anns = coco.loadAnns(annIds)
        for ann in anns:
            keypoints = np.array(ann['keypoints']).reshape(17,3)
            visible_keypoints = {}
            for index, (x, y, v) in enumerate(keypoints): 
                if v == 2:
                    image = cv2.circle(image, (x,y), radius=3, color=(255, 255, 255), thickness=-1)
                    visible_keypoints[index + 1] = (x, y)
            for p1, p2 in skeleton:
                if p1 in visible_keypoints and p2 in visible_keypoints: 
                    image = cv2.line(image, visible_keypoints[p1], visible_keypoints[p2], color=(255, 255, 255), thickness=2)
        new_name = filepath.format(file_split, i.split('/')[-1])
        cv2.imwrite(new_name, image)

In [ ]:
for split in ['train', 'val']:
    filepath = 'data/skeleton_black/{}/{}'
    for file in os.listdir('data/skeleton/{}'.format(split)):
        image = cv2.imread('data/skeleton/{}/{}'.format(split, file))
        inv_image = np.invert(image)
        new_name = filepath.format(split, file)
        cv2.imwrite(new_name, inv_image)
plt.imshow(inv_image)